In [1]:
!pip install transformers sentencepiece
!pip install sentence-transformers
!pip install nltk
!pip install PyMuPDF
!git clone https://github.com/allenai/PeerRead.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import os, re, json, glob, torch
import fitz  # PyMuPDF
import nltk
from google.colab import files
from nltk.sentiment import SentimentIntensityAnalyzer
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

nltk.download('vader_lexicon')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
uploaded = files.upload()
pdf_path = f"/content/{next(iter(uploaded))}"

def extract_text_from_pdf(filepath):
    doc = fitz.open(filepath)
    return "\n".join([page.get_text() for page in doc])

raw_text = extract_text_from_pdf(pdf_path)

Saving Handwritten Equation Solver 1.pdf to Handwritten Equation Solver 1.pdf


In [5]:
def split_subsections(text):
    return re.split(r'\n(?=\d?\s?[A-Z][a-z]+.*?:)', text)

def split_paragraphs_cleaned(subsection_text):
    paragraphs = subsection_text.split("\n")
    cleaned = []
    for p in paragraphs:
        p = p.strip()
        if len(p) < 50:
            continue
        if re.search(r'^\s*(Figure|Table|Eq\.|Equation)\s*\d+', p, re.IGNORECASE):
            continue
        if re.search(r'\b(fig\.|equation|table)\b', p, re.IGNORECASE) and len(p.split()) < 15:
            continue
        cleaned.append(p)
    return cleaned


In [7]:
# Summarizer using BART
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)
summarizer_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Suggestion using google Flan-T5 model
suggestion_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
suggestion_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Plagiarism check
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Bias check
sentiment_analyzer = SentimentIntensityAnalyzer()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
peer_paragraphs = []
review_dir = "/content/PeerRead/data/acl_2017/train/reviews/"
review_files = glob.glob(os.path.join(review_dir, "*.json"))

for file in review_files:
    if len(peer_paragraphs) >= 5000:
        break
    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
            abstract = data.get("abstract", "").strip()
            if len(abstract.split()) > 20:
                peer_paragraphs.append(abstract)
    except:
        continue

print(f"\n✅ Loaded {len(peer_paragraphs)} review abstracts from PeerRead.")

peer_embeddings = (
    embedder.encode(peer_paragraphs, convert_to_tensor=True)
    if peer_paragraphs else None
)


✅ Loaded 122 review abstracts from PeerRead.


In [10]:
def summarize_text(text):
    inputs = summarizer_tokenizer(text, return_tensors="pt", truncation=True, max_length=1024).to(device)
    with torch.no_grad():
        output_ids = summarizer_model.generate(inputs["input_ids"], max_length=200)
    return summarizer_tokenizer.decode(output_ids[0], skip_special_tokens=True)

def generate_suggestion(text):
    prompt = prompt = (
        "You are an academic writing assistant reviewing the following section of a research paper.\n"
        "Your job is to provide suggestions to improve the writing, structure, and clarity.\n"
        "Focus on identifying awkward phrasing, missing citations, unclear argument flow, and overly long sentences.\n\n"
        "Research Paper Section:\n"
        f"{text}\n\n"
        "Write improvement suggestions:"
    )
    inputs = suggestion_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        output_ids = suggestion_model.generate(inputs["input_ids"], max_length=200)
    return suggestion_tokenizer.decode(output_ids[0], skip_special_tokens=True)

def check_bias(para):
    score = sentiment_analyzer.polarity_scores(para)
    return score['compound'], "yes" if abs(score['compound']) > 0.2 else "no"

def check_plagiarism(para):
    if peer_embeddings is None or peer_embeddings.shape[0] == 0:
        return 0.0, "no"
    query_vec = embedder.encode(para, convert_to_tensor=True)
    sim_scores = util.pytorch_cos_sim(query_vec, peer_embeddings)
    max_sim = torch.max(sim_scores).item()
    return max_sim, "yes" if max_sim > 0.75 else "no"

In [12]:
common_sections = {
    "abstract": "Abstract",
    "introduction": "Introduction",
    "related work": "Related Work",
    "background": "Background",
    "literature review": "Literature Review",
    "method": "Methodology",
    "implementation": "Implementation",
    "experiment": "Experiments",
    "result": "Results",
    "discussion": "Discussion",
    "conclusion": "Conclusion"
}

def map_to_standard_title(title):
    title_lower = title.lower()
    for key, std_title in common_sections.items():
        if key in title_lower:
            return std_title
    return title.strip()[:50]


In [13]:
subsections = split_subsections(raw_text)

for i, subsection in enumerate(subsections):
    lines = subsection.strip().split('\n')
    if len(lines) < 2:
        continue

    title = lines[0].strip()
    body = "\n".join(lines[1:]).strip()
    mapped_title = map_to_standard_title(title)
    paragraphs = split_paragraphs_cleaned(body)

    print(f"\n\n===== Subsection {i+1}: {mapped_title} =====\n")

    summary = summarize_text(body)
    suggestion = generate_suggestion(body)

    print("Summary:\n", summary)
    print("\nImprovement Suggested:\n", suggestion)

    for j, para in enumerate(paragraphs):
        bias_score, bias_flag = check_bias(para)
        plag_score, plag_flag = check_plagiarism(para)

        print(f"\nParagraph {j+1}:\n{para}")
        print(f"Plagiarism Score: {plag_score:.4f}, plagiarized = {plag_flag}")
        print(f"Bias Score: {bias_score:.4f}, biased = {bias_flag}")



===== Subsection 1: Handwritten Equation Solver =====

Summary:
 Deep learning-based approach to automatically recognize and solve handwritten equations. By combining computer vision, convolutional neural networks, and symbolic.computation, the system improves efficiency and accuracy. The system shows promise in aiding students, educators, and researchers in mathematical problem-solving.

Improvement Suggested:
 The paper is a challenge to digitize due to variations in handwriting, overlapping symbols, and complex notations.

Paragraph 1:
challenging to digitize due to variations in handwriting,
Plagiarism Score: 0.3298, plagiarized = no
Bias Score: 0.1531, biased = no

Paragraph 2:
overlapping symbols, and complex notations. Current Optical
Plagiarism Score: 0.1787, plagiarized = no
Bias Score: 0.0000, biased = no

Paragraph 3:
mathematical expressions, leading to inaccuracies. This project
Plagiarism Score: 0.2444, plagiarized = no
Bias Score: 0.0000, biased = no

Paragraph 4:
deve